In [12]:
import math
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
class Perceptron:
    def __init__(self,weights,bias):
        """
        inputs : a vector of inputs 
        weights : a vector of weights 
        output : the provided output
        """
        self.weights = np.array(weights)
        self.bias = bias
    
    def activation_function(self, x):
        # sigmoid for training
        return 1/(1+math.exp(-x))
    
    def Hard_activation_function(self, x):
        # step function for prediction (purely academic)
        return 1 if x > 0 else 0
    
    def predict(self,inputs):
        x = np.dot(self.weights, inputs) + self.bias
        return self.activation_function(x)

In [3]:
class PerceptronLayer:
    def __init__(self, layer_id, n_inputs, n_neurons):
        self.layer_id = layer_id
        self.n_inputs = n_inputs
        self.n_neurons = n_neurons
        
        # Vectorized weights and biases
        self.weights = np.random.randn(n_neurons, n_inputs) * 0.1
        self.biases = np.zeros((n_neurons, 1))
        
        # values for backprop
        self.z = None
        self.a = None
        self.inputs = None

    def activation(self, z):
        # Sigmoid
        return 1 / (1 + np.exp(-z))
    
    def activation_derivative(self, z):
        a = self.activation(z)
        return a * (1 - a)
    
    def forward(self, inputs):
        self.inputs = inputs
        self.z = np.dot(self.weights, inputs) + self.biases
        self.a = self.activation(self.z)
        return self.a

In [4]:
class FeedForwardNeuralNetwork:
    def __init__(self, n_inputs, n_outputs, hidden_layers):
        self.structure = [n_inputs] + hidden_layers + [n_outputs]
        self.layers = []
        
        for i in range(1, len(self.structure)):
            layer = PerceptronLayer(i-1, self.structure[i-1], self.structure[i])
            self.layers.append(layer)
    
    def forward(self, x):
        a = x
        for layer in self.layers:
            a = layer.forward(a)
        return a
    
    def backward(self,y_true,eta = 0.1):
        n_layers = len(self.layers)
        deltas = [None]*n_layers

        output_layer = self.layers[-1]
        deltas[-1] = (output_layer.a-y_true)*output_layer.activation_derivative(output_layer.z) # this is delta_k for the output layer

        for l in range(n_layers-2,-1,-1):
            layer=self.layers[l]
            next_layer = self.layers[l+1]
            deltas[l] = np.dot(next_layer.weights.T,deltas[l+1])*layer.activation_derivative(layer.z)
        
        for l in range(n_layers):
            layer = self.layers[l]
            a_prev = self.layers[l-1].a if l>0 else layer.inputs
            layer.weights -= eta * np.dot(deltas[l],a_prev.T)
            layer.biases -= eta * deltas[l]
    
    def train(self, X_train, Y_train, epochs=1000, lr=0.1):
        for epoch in range(epochs):
            loss = 0
            for x, y in zip(X_train, Y_train):
                x = np.array(x).reshape(-1,1)
                y = np.array(y).reshape(-1,1)
                output = self.forward(x)
                loss += np.sum((output - y)**2)/2
                self.backward(y, eta=lr)
            if epoch % 500 == 0:
                print(f"Epoch {epoch}, Loss: {loss:.4f}")

In [5]:
net = FeedForwardNeuralNetwork(3, 3, [math.comb(3,k) for k in range(1,4)])
X = [[0,0,0],[0,1,1],[1,0,1],[1,1,1]]
Y = [[0],[1],[2],[2]]
net.train(X, Y, epochs=5000, lr=0.5)

Epoch 0, Loss: 7.3403
Epoch 500, Loss: 4.5059
Epoch 1000, Loss: 4.5030
Epoch 1500, Loss: 4.5020
Epoch 2000, Loss: 4.5015
Epoch 2500, Loss: 4.5012
Epoch 3000, Loss: 4.5010
Epoch 3500, Loss: 4.5009
Epoch 4000, Loss: 4.5007
Epoch 4500, Loss: 4.5007


In [6]:
# Define column names (from UCI dataset description)
columns = [
    'status', 'duration', 'credit_history', 'purpose', 'credit_amount',
    'savings', 'employment', 'installment_rate', 'personal_status_sex',
    'other_debtors', 'residence_since', 'property', 'age', 'other_installment_plans',
    'housing', 'number_credits', 'job', 'people_liable', 'telephone', 'foreign_worker', 'target'
]

data = pd.read_csv("german_credit_data/german.data", sep=' ', header=None, names=columns)

data.head()

,status,duration,credit_history,purpose,credit_amount,savings,employment,installment_rate,personal_status_sex,other_debtors,...,property,age,other_installment_plans,housing,number_credits,job,people_liable,telephone,foreign_worker,target
0,A11,6,A34,A43,1169,A65,A75,4,A93,A101,...,A121,67,A143,A152,2,A173,1,A192,A201,1
1,A12,48,A32,A43,5951,A61,A73,2,A92,A101,...,A121,22,A143,A152,1,A173,1,A191,A201,2
2,A14,12,A34,A46,2096,A61,A74,2,A93,A101,...,A121,49,A143,A152,1,A172,2,A191,A201,1
3,A11,42,A32,A42,7882,A61,A74,2,A93,A103,...,A122,45,A143,A153,1,A173,2,A191,A201,1
4,A11,24,A33,A40,4870,A61,A73,3,A93,A101,...,A124,53,A143,A153,2,A173,2,A191,A201,2


In [7]:
data['target'] = data['target'].apply(lambda x: 1 if x == 1 else 0)

In [8]:
categorical_cols = [
    'status', 'credit_history', 'purpose', 'savings', 'employment',
    'personal_status_sex', 'other_debtors', 'property',
    'other_installment_plans', 'housing', 'job', 'telephone', 'foreign_worker'
]

numerical_cols = [col for col in data.columns if col not in categorical_cols + ['target']]


In [9]:
data = pd.get_dummies(data, columns=categorical_cols, drop_first=True)

In [ ]:


X = data.drop('target', axis=1).values
y = data['target'].values.reshape(-1,1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

n_inputs = X_train.shape[1]
n_outputs = 1
hidden_layers = [16,16]

nn = FeedForwardNeuralNetwork(n_inputs, n_outputs, hidden_layers)

# Train
nn.train(X_train, y_train, epochs=2000, lr=0.01)


Epoch 0, Loss: 87.0522
